In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import sys

sys.path.append("/nas/ckgfs/users/minhpham/workspace/seed")

In [11]:
from seed.pipeline.pipeline import Pipeline

In [ ]:
from seed.retriever import Retriever

retriever = Retriever.get("dpr")

Attempting to initialize pre-built index wikipedia-dpr.
/nas/ckgfs/users/minhpham/.cache/pyserini/indexes/index-wikipedia-dpr-20210120-d1b9e6.c28f3a56b2dfcef25bf3bf755c264d04 already exists, skipping download.
Initializing wikipedia-dpr...


Some weights of the model checkpoint at facebook/dpr-question_encoder-multiset-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Attempting to initialize pre-built index wikipedia-dpr-multi-bf.
/nas/ckgfs/users/minhpham/.cache/pyserini/indexes/dindex-wikipedia-dpr_multi-bf-20200127-f403c3.29eb39fe0b00a03c36c0eeae4c24f775 already exists, skipping download.
Initializing wikipedia-dpr-multi-bf...


In [ ]:
from seed.verifier import Verifier

verifier = Verifier.get("tapex")

In [ ]:
from seed.extractor import Extractor

extractor = Extractor.get("unifiedqa")

In [ ]:
pipeline = Pipeline(retriever, verifier, extractor)

In [ ]:
import pandas as pd
import jsonlines

df = pd.DataFrame(list(jsonlines.open("../data/totto/totto_dev_data_augmented.jsonl")))

In [ ]:
import json

for idx, obj in df.iterrows():
    if not obj["highlighted_cells"]:
        continue
    table = pd.DataFrame(json.loads(obj["table"]))
    rows = [x[0] for x in obj["highlighted_cells"]]
    most_dominant_row = max(set(rows), key=rows.count)
    cols = [x[1] for x in obj["highlighted_cells"] if x[0] == most_dominant_row]
    table = table.iloc[[most_dominant_row], ].reset_index().drop("index", axis=1)
    print(table)

    result = pipeline.run(table)
    break